In [8]:
 # Source code: https://meshlogic.github.io/posts/jupyter/curve-fitting/parametric-curve-fitting/

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
import numpy as np
from numpy.polynomial import polynomial as pl
import os
import math
import numpy as np
from numpy import linalg, linspace, zeros
from math import ceil, log
from matplotlib import cm


In [9]:
#function to read a file with coordinates and extract the x, y, and z coordinates for each file
def read_xyz_file(filename):
    with open(filename, 'r') as file:
        
        coordinates = [list(map(float, line.split(","))) for line in file]
        # Splitting the line by whitespace and converting values to float
    x_coordinates, y_coordinates, z_coordinates = zip(*coordinates)

    return x_coordinates, y_coordinates, z_coordinates

# This function finds distance between two points in 3D space
def finddistance(x,y,z,x1,y1,z1):
    return math.sqrt((x-x1)**2 + (y-y1)**2 + (z-z1)**2)


In [10]:
def uniform_param(P):
    u = linspace(0, 1, len(P))
    return u

def chordlength_param(P):
    u = generate_param(P, alpha=1.0)
    return u

def centripetal_param(P):
    u = generate_param(P, alpha=0.5)
    return u

# This function generates parameters for a curve based on a given power
def generate_param(P, alpha):
    n = len(P)
    u = zeros(n)
    u_sum = 0
    for i in range(1,n):
        u_sum += linalg.norm(P[i,:]-P[i-1,:])**alpha
        u[i] = u_sum
    return u/max(u)

# This function performs the golden section search to find minimum of a function within an interval
def find_min_gss(f, a, b, eps=1e-4):

    R = 0.61803399
    n_iter = int(ceil(-2.0780869 * log(eps/abs(b-a))))
    c = b - (b-a)*R
    d = a + (b-a)*R

    for i in range(n_iter):
        if f(c) < f(d):
            b = d
        else:
            a = c
        c = b - (b-a)*R
        d = a + (b-a)*R
    return (b+a)/2

# This function generates iterative parameters for curve fitting and visualizes the error in approximation
def iterative_param(P, u, fxcoeff, fycoeff, fzcoeff, fig_ax):

    global iter_i, plt_color
    u_new = u.copy()
    f_u = zeros(3)

    def calc_s(u):
        f_u[0] = np.polyval(fxcoeff, u)
        f_u[1] = np.polyval(fycoeff, u)
        f_u[2] = np.polyval(fzcoeff, u)
        s_u = linalg.norm(P[i]-f_u)
        return s_u

    for i in range(1, len(u)-1):
        u_new[i] = find_min_gss(calc_s, u[i-1], u[i+1])
        u_samp = linspace(u[i-1], u[i+1], 25)
        x = np.polyval(fxcoeff, u_samp)
        y = np.polyval(fycoeff, u_samp)
        z = np.polyval(fzcoeff, u_samp)
        residual = P[i] - np.array([x,y,z]).T
        s_u_samp = [linalg.norm(residual[j]) for j in range(len(u_samp))]
        fig_ax.plot(u_samp, s_u_samp, color=plt_color[iter_i], alpha=plt_alpha)
        fig_ax.plot(u_new[i], calc_s(u_new[i]), 'o', color=plt_color[iter_i], alpha=plt_alpha)

    return u_new

In [11]:
#splits the coordinates of phosphorous atoms into two sets of coordinates - x, y, and z coords for each helix
def process_coordinates(directory, filename):
    filepath = os.path.join(directory, filename)
    xs, ys, zs = read_xyz_file(filepath)

    #print_coords('x', xs)
    #print_coords('y', ys)
    #print_coords('z', zs)

    middle_elementx = len(xs) // 2

    endindex = middle_elementx
    if len(xs) % 2 == 1 :
        distance1 = finddistance(xs[middle_elementx], ys[middle_elementx], zs[middle_elementx], xs[middle_elementx-1], ys[middle_elementx-1], zs[middle_elementx-1])
        distance2 = finddistance(xs[middle_elementx], ys[middle_elementx], zs[middle_elementx], xs[middle_elementx+1], ys[middle_elementx+1], zs[middle_elementx+1])
        if distance1 < distance2:
            endindex += 1

    x, x1 = split_and_print_coords('x', xs, endindex)
    y, y1 = split_and_print_coords('y', ys, endindex)
    z, z1 = split_and_print_coords('z', zs, endindex)

    return x, y, z, x1, y1, z1

def print_coords(name, coords):
    print(f'Number of {name} coordinates: {len(coords)}')
    print(f'{name} coordinates: {coords}')

def split_and_print_coords(name, coords, index):
    first_half = coords[:index]
    second_half = coords[index:]

    #print_coords(f'{name} first half', first_half)
    #print_coords(f'{name} second half', second_half)

    return first_half, second_half


In [12]:
polydeg=9
max_iter=20
eps=1e-3
def prepare_plot(x, y, z):
    P = np.column_stack((x, y, z))

    w = np.ones(len(P))
    w[0] = w[-1] = 1e6  # Set high weights for first and last points

    f_u = np.zeros([len(P), 3])
    uu = np.linspace(0, 1, 100)
    f_uu = np.zeros([len(uu), 3])

    fig, axs = plt.subplots(3, 3, figsize=(15, 15))

    for i, (ax, title, xlabel, ylabel, plot_data, aspect) in enumerate([
        (axs[0, 0], 'View X-Y', 'x', 'y', (P[:, 0], P[:, 1]), 'equal'),
        (axs[0, 1], 'View X-Z', 'x', 'z', (P[:, 0], P[:, 2]), 'equal'),
        (axs[0, 2], 'View Y-Z', 'y', 'z', (P[:, 1], P[:, 2]), 'equal'),
        (axs[1, :], 'Local minimization of approximation error $s(u_i)$ for each $u_i$', '$u_i$', '$s(u_i)$', None, None),
        (axs[2, :2], 'Approximation error $S$ for each iteration', 'Iteration', '$S$', None, None)
    ]):
        if isinstance(ax, np.ndarray):
            for single_ax in ax.flatten():
                if plot_data:
                    single_ax.plot(*plot_data, 'ks', label='Knot points P')
                single_ax.set_title(title)
                single_ax.set_xlabel(xlabel); single_ax.set_ylabel(ylabel);
                single_ax.margins(.1, .1)
                single_ax.grid()
                if aspect:
                    single_ax.set_aspect(aspect, 'datalim')
        else:
            if plot_data:
                ax.plot(*plot_data, 'ks', label='Knot points P')
            ax.set_title(title)
            ax.set_xlabel(xlabel); ax.set_ylabel(ylabel);
            ax.margins(.1, .1)
            ax.grid()
            if aspect:
                ax.set_aspect(aspect, 'datalim')

    plt_color = cm.rainbow(np.linspace(1, 0, max_iter))
    return fig, axs, plt_color, w


In [ ]:
f = open("coefficients.txt", "w")
directory = r"C:\Users\magne\Downloads\DNA_Coords\DNA_Coords"
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
            print(filename)
            x, y, z, x1, y1, z1 = process_coordinates(directory, filename)
            
            print(filename)
            fig, axs, plt_color, w = prepare_plot(x, y, z)
            prepare_plot(x1, y1, z1)

            P = zeros([len(x),3])
            P[:,0] = x
            P[:,1] = y
            P[:,2] = z

            P1 = zeros([len(x1),3])
            P1[:,0] = x1
            P1[:,1] = y1
            P1[:,2] = z1
            plt_alpha = 0.6
            w = np.ones(len(P))
            w1 = np.ones(len(P1))
            polydeg = 9
            w[0] = w[-1] = 1e6  # Set high weights for first and last point
            max_iter = 20
            u, u1 = centripetal_param(P), centripetal_param(P1)
            S_hist = []
            n = len(P)
            
            #-------------------------------------------------------------------------------
            # Init variables
            #-------------------------------------------------------------------------------
            f_u = zeros([n,3])
            uu = linspace(0,1,100)
            f_uu = zeros([len(uu),3])

            for iter_i in range(max_iter):
                print(iter_i)
                if iter_i > 0:
                    u = iterative_param(P, u, fxcoeff, fycoeff, fzcoeff, axs[1,0])
                    
                    u1 = iterative_param(P1, u1, fxcoeff, fycoeff, fzcoeff, axs[1,0])
                    

                fxcoeff = pl.polyfit(u, P[:,0], polydeg, w=w)
                fycoeff = pl.polyfit(u, P[:,1], polydeg, w=w)
                fzcoeff = pl.polyfit(u, P[:,2], polydeg, w=w)
                fx1coeff = pl.polyfit(u1, P1[:,0], polydeg, w=w1)
                fy1coeff = pl.polyfit(u1, P1[:,1], polydeg, w=w1)
                fz1coeff = pl.polyfit(u1, P1[:,2], polydeg, w=w1)

                f_u[:,0] = pl.polyval(u, fxcoeff)
                f_u[:,1] = pl.polyval(u, fycoeff)
                f_u[:,2] = pl.polyval(u, fzcoeff)
                f_uu[:,0] = pl.polyval(uu, fxcoeff)
                f_uu[:,1] = pl.polyval(uu, fycoeff)
                f_uu[:,2] = pl.polyval(uu, fzcoeff)

                hp = axs[0, 0].plot(f_uu[:,0], f_uu[:,1], color=plt_color[iter_i], alpha=plt_alpha)
                hp = axs[0, 1].plot(f_uu[:,0], f_uu[:,2], color=plt_color[iter_i], alpha=plt_alpha)
                hp = axs[0, 2].plot(f_uu[:,1], f_uu[:,2], color=plt_color[iter_i], alpha=plt_alpha)


                if iter_i == 0:
                    for i in range(1, len(u)-1):
                        axs[1, 0].plot(u[i], linalg.norm(P[i]-f_u[i]), 'o', color=plt_color[iter_i], alpha=plt_alpha)

                S = 0
                for j in range(len(u)):
                    S += w[j] * linalg.norm(P[j] - f_u[j])

                #ax[4].bar(iter_i, S, width=0.6, color=plt_color[iter_i], alpha=plt_alpha)
                S_hist.append(S)
            
                
                if iter_i > 0 and (S_hist[iter_i-1] / S_hist[iter_i] - 1) < eps:
                    print(fxcoeff)
                    try:
                        with open("coefficients.txt", "a") as f:
                            f.write(f"{filename}\n{fxcoeff}\n{fycoeff}\n{fzcoeff}\n{fx1coeff}\n{fy1coeff}\n{fz1coeff}\n")
                        break
                    except:
                        print("An error occurred trying to write the file.")


3vd1_M-3vd1_P DNA.txt
3vd1_M-3vd1_P DNA.txt
0
1
2
[ 4.91040000e+01  5.97878954e+01 -1.36729244e+03  1.66699270e+04
 -1.10751521e+05  3.74376014e+05 -6.86292615e+05  6.96110671e+05
 -3.67977628e+05  7.91795844e+04]
3vd2_E-3vd2_F DNA.txt
3vd2_E-3vd2_F DNA.txt
0
1
[ 5.82310000e+01 -5.63203561e+01  1.20869166e+03 -6.51115067e+03
  1.26886369e+04 -1.11376841e+03 -3.59926776e+04  6.31630191e+04
 -4.63623048e+04  1.29829481e+04]
3vd2_G-3vd2_H DNA.txt
3vd2_G-3vd2_H DNA.txt
0
1
[ 5.85780000e+01 -1.86751366e+02  2.90035373e+03 -2.59742777e+04
  1.21843463e+05 -3.24599727e+05  5.15707086e+05 -4.85338594e+05
  2.49679655e+05 -5.40462133e+04]
3vd2_K-3vd2_L DNA.txt
3vd2_K-3vd2_L DNA.txt
0
1
[ 3.63240000e+01 -1.90698816e+02  3.58574522e+03 -3.15797351e+04
  1.48211291e+05 -4.12257797e+05  7.00080407e+05 -7.09301109e+05
  3.92625263e+05 -9.11930557e+04]
3vke_R-3vke_S DNA.txt
3vke_R-3vke_S DNA.txt
0
1
[-1.23130000e+01 -7.42448038e+01  1.72187481e+03 -1.60172659e+04
  8.48090358e+04 -2.65351148e+05  4.9

C:\Users\magne\AppData\Local\Temp\ipykernel_25336\3582630444.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(3, 3, figsize=(15, 15))


1
[ 4.14600000e+00  2.56412579e+02 -3.16461788e+02  1.42940828e+03
 -1.20209220e+05  6.65642151e+05 -1.49665682e+06  1.67105883e+06
 -9.22545847e+05  2.01282997e+05]
3waa_I-3waa_J DNA.txt
3waa_I-3waa_J DNA.txt
0
1
[-4.38100000e+00 -2.51612440e+02  2.71849537e+02 -1.25018993e+03
  1.20244462e+05 -6.67826485e+05  1.50291228e+06 -1.67892234e+06
  9.27268462e+05 -2.02387678e+05]
3wkj_I-3wkj_J DNA.txt
3wkj_I-3wkj_J DNA.txt
0
1
[-4.81800000e+00 -2.78776588e+02  1.55773333e+03 -1.72993422e+04
  2.14985775e+05 -9.75188536e+05  2.08058298e+06 -2.30509096e+06
  1.29028785e+06 -2.89509063e+05]
3wtp_I-3wtp_J DNA.txt
3wtp_I-3wtp_J DNA.txt
0
1
[-2.65300000e+00  3.26611334e+02 -3.83459574e+02  2.04993494e+03
 -1.20307904e+05  6.35432881e+05 -1.37707174e+06  1.48342011e+06
 -7.88329373e+05  1.64805024e+05]
3wts_D-3wts_E DNA.txt
3wts_D-3wts_E DNA.txt
0
1
[ 1.41950000e+01  1.04949325e+02 -1.82376767e+03  2.17883918e+04
 -1.34386840e+05  4.45280963e+05 -8.40114121e+05  9.08549961e+05
 -5.25153000e+05  1.

[ 2.81000000e-01 -3.55023704e+00 -2.31531538e+02  3.97548168e+03
 -2.35339955e+04  7.48593322e+04 -1.37015333e+05  1.43966072e+05
 -8.06748823e+04  1.86815287e+04]
4bqa_E-4bqa_F DNA.txt
4bqa_E-4bqa_F DNA.txt
0
1
[-3.28060000e+01  1.30366838e+02 -1.14819380e+03  6.01713511e+03
 -1.15473558e+04 -9.31419426e+03  7.44205815e+04 -1.23261857e+05
  8.94259624e+04 -2.47129642e+04]
4bqa_H-4bqa_I DNA.txt
4bqa_H-4bqa_I DNA.txt
0
1
[-2.34600000e+00  6.49629989e+00 -8.51794851e+02  8.03292725e+03
 -4.00278077e+04  1.18637025e+05 -2.11168422e+05  2.20257682e+05
 -1.23837419e+05  2.89425181e+04]
4cn2_A-4cn2_B DNA.txt
4cn2_A-4cn2_B DNA.txt
0
1
2
3
4
5
6
7
8
9
[-2.38300000e+00  7.62550328e+01 -2.84553728e+03  3.73095083e+04
 -2.32403835e+05  7.97111324e+05 -1.57537256e+06  1.78383956e+06
 -1.07372138e+06  2.66031163e+05]
4cn3_E-4cn3_F DNA.txt
4cn3_E-4cn3_F DNA.txt
0
1
2
3
4
5
[-1.27540000e+01  3.70267946e+01  3.55945131e+02  2.10803951e+03
 -3.95817936e+04  2.04409375e+05 -5.19107106e+05  7.04800933e+0

1
[ 2.78870000e+01 -5.61068199e+00  5.74738514e+02 -9.91520134e+03
  7.19088358e+04 -2.56620371e+05  5.04331124e+05 -5.58194241e+05
  3.26201979e+05 -7.82794286e+04]
4ecz_P-4ecz_T DNA.txt
4ecz_P-4ecz_T DNA.txt
0
1
2
3
[ 2.76890000e+01  1.51426342e+01 -2.30734210e+01 -2.46600358e+03
  2.36017292e+04 -8.62315999e+04  1.63345803e+05 -1.71394253e+05
  9.48158726e+04 -2.16589792e+04]
4ed0_P-4ed0_T DNA.txt
4ed0_P-4ed0_T DNA.txt
0
1
[ 2.77540000e+01  3.11905034e+01 -5.11586770e+02  2.00845959e+03
  6.87915070e+03 -5.72963940e+04  1.44813707e+05 -1.79759787e+05
  1.11226970e+05 -2.73858614e+04]
4ed1_P-4ed1_T DNA.txt
4ed1_P-4ed1_T DNA.txt
0
1
[ 2.73650000e+01  7.07799625e+00  5.70592447e+02 -1.11206815e+04
  7.94596716e+04 -2.76330852e+05  5.29144575e+05 -5.71266641e+05
  3.26147552e+05 -7.66089860e+04]
4ed2_P-4ed2_T DNA.txt
4ed2_P-4ed2_T DNA.txt
0
1
[ 2.77720000e+01  1.10650660e+01  1.74154033e+02 -6.28483723e+03
  5.56933781e+04 -2.15888448e+05  4.43798362e+05 -5.05059766e+05
  3.00416541e+05

...

1
[-1.43040000e+01 -1.38458786e+02  4.24559047e+03 -3.87185330e+04
  1.69229659e+05 -4.22358921e+05  6.36590109e+05 -5.74180903e+05
  2.85384824e+05 -6.00686757e+04]
4k4g_N-4k4g_P DNA.txt
4k4g_N-4k4g_P DNA.txt
0
1
[-1.42670000e+01 -5.05611902e+02  1.37400737e+04 -1.20765504e+05
  5.33951226e+05 -1.36401676e+06  2.10107746e+06 -1.92548666e+06
  9.66463777e+05 -2.04471524e+05]
4k4h_B-4k4h_D DNA.txt
4k4h_B-4k4h_D DNA.txt
0
1
[ 2.87100000e+00 -6.19812680e+01 -7.24788786e+02  1.41822920e+04
 -9.71797119e+04  3.30578950e+05 -6.22838465e+05  6.62694140e+05
 -3.73419897e+05  8.67542501e+04]
4k4h_F-4k4h_H DNA.txt
4k4h_F-4k4h_H DNA.txt
0
1
[-2.00160000e+01  4.90554510e+02 -3.22641091e+03  9.76036925e+03
 -1.20742865e+04 -1.28672848e+04  6.54140861e+04 -9.11119802e+04
  5.78426666e+04 -1.42296571e+04]
4k4h_J-4k4h_L DNA.txt
4k4h_J-4k4h_L DNA.txt
0
1
[-2.94120000e+01  1.78220427e+02  1.30352314e+03 -2.19735662e+04
  1.07510608e+05 -2.74495964e+05  4.09535884e+05 -3.59492193e+05
  1.71937555e+05 -3.

1
[ 3.50630000e+01  1.68066358e+02 -8.24235060e+03  1.05352607e+05
 -6.14851918e+05  1.91880840e+06 -3.42168468e+06  3.50034920e+06
 -1.91346804e+06  4.33574758e+05]
4m9j_D-4m9j_T DNA.txt
4m9j_D-4m9j_T DNA.txt
0
1
[ 3.51190000e+01 -1.45663312e+02  1.34163441e+03  2.70146652e+03
 -7.75284423e+04  3.48011278e+05 -7.21361964e+05  7.85258359e+05
 -4.35286996e+05  9.70112772e+04]
4m9l_D-4m9l_T DNA.txt
4m9l_D-4m9l_T DNA.txt
0
1
[-4.24000000e+00  2.39762195e+02 -8.72170847e+03  7.12466487e+04
 -2.38634895e+05  3.55397415e+05 -1.25091022e+05 -2.73443518e+05
  3.26545042e+05 -1.07552865e+05]
4m9n_D-4m9n_T DNA.txt
4m9n_D-4m9n_T DNA.txt
0
1
[ 3.50280000e+01 -9.46613712e+00 -3.14365983e+03  5.59790091e+04
 -3.89810999e+05  1.36210095e+06 -2.63199938e+06  2.86185576e+06
 -1.64212271e+06  3.87156725e+05]
4mf2_D-4mf2_T DNA.txt
4mf2_D-4mf2_T DNA.txt
0
1
[ 3.73260000e+01  8.16536315e-01 -3.91043380e+03  5.96279971e+04
 -3.70696921e+05  1.18519687e+06 -2.13008611e+06  2.18129083e+06
 -1.19106190e+06  2.

1
[ 2.85290000e+01 -1.68715653e+02  2.47626082e+03 -2.17632030e+04
  9.64465165e+04 -2.21573307e+05  2.70495504e+05 -1.66622221e+05
  4.02978709e+04  4.13500426e+02]
4nod_K-4nod_L DNA.txt
4nod_K-4nod_L DNA.txt
0
1
[ 3.21190000e+01 -1.72896205e+02  1.77715386e+03 -1.26315028e+04
  4.56701183e+04 -7.28017099e+04  2.23761550e+04  7.04924344e+04
 -8.07554824e+04  2.60400917e+04]
4nqa_E-4nqa_F DNA.txt
4nqa_E-4nqa_F DNA.txt
0
1
[-9.94900000e+00  2.12944600e+02 -3.23468297e+03  2.85191920e+04
 -1.30604364e+05  3.34738886e+05 -5.07783400e+05  4.58027579e+05
 -2.28845757e+05  4.89893212e+04]
4nqa_L-4nqa_M DNA.txt
4nqa_L-4nqa_M DNA.txt
0
1
[    44.507          32.34985283   -815.53620549   6080.25250087
 -15382.44821652  14461.66574628  -4737.58983399   7828.65172434
 -13415.74517883   5963.09461052]
4nw3_B-4nw3_C DNA.txt
4nw3_B-4nw3_C DNA.txt
0
1
[ 3.50640000e+01 -6.88772321e+01  5.88523883e+02 -2.42140326e+03
 -1.89313839e+03  3.47646686e+04 -9.59733482e+04  1.24423760e+05
 -7.99545783e+04  2.

1
[ 3.72900000e+01  6.24056228e+01 -5.63132940e+03  7.52560668e+04
 -4.36778551e+05  1.33650040e+06 -2.32874180e+06  2.32991357e+06
 -1.24934060e+06  2.78763208e+05]
4php_D-4php_T DNA.txt
4php_D-4php_T DNA.txt
0
1
[ 3.01200000e+01 -4.83621811e+01 -1.53221523e+03  2.77791865e+04
 -1.77834008e+05  5.66992149e+05 -1.00285710e+06  1.00405346e+06
 -5.34535816e+05  1.17987239e+05]
4ppx_D-4ppx_T DNA.txt
4ppx_D-4ppx_T DNA.txt
0
1
2
3
[ 3.47100000e+01  8.60653559e+01 -5.75206674e+03  8.01948419e+04
 -4.93701222e+05  1.59735891e+06 -2.92336056e+06  3.04908813e+06
 -1.69156971e+06  3.87658645e+05]
4pw5_C-4pw5_D DNA.txt
4pw5_C-4pw5_D DNA.txt
0
1
[-1.70100000e+00 -2.10721585e+02  4.55354741e+03 -4.85814224e+04
  2.66229587e+05 -8.30927856e+05  1.53054859e+06 -1.64244995e+06
  9.47640736e+05 -2.26834191e+05]
4pw5_G-4pw5_H DNA.txt
4pw5_G-4pw5_H DNA.txt
0
1
[-1.26590000e+01 -2.80287976e+02  7.12451619e+03 -7.88207135e+04
  4.37397105e+05 -1.36551246e+06  2.50076160e+06 -2.66092584e+06
  1.52085422e+06

1
[ 3.36000000e+00 -3.32228249e+02  7.47725508e+03 -6.41429969e+04
  2.78824470e+05 -6.58529697e+05  8.65028282e+05 -6.13882695e+05
  2.07604171e+05 -2.20657447e+04]
4ria_I-4ria_L DNA.txt
4ria_I-4ria_L DNA.txt
0
1
[ 6.41300000e+01 -1.46875714e+02  4.18515125e+03 -5.21316898e+04
  3.10582778e+05 -9.77835296e+05  1.71633207e+06 -1.68904400e+06
  8.71895381e+05 -1.83887810e+05]
4rib_U-4rib_W DNA.txt
4rib_U-4rib_W DNA.txt
0
1
[ 1.43630000e+01 -1.89254765e+02  2.22879900e+03 -1.11880815e+04
  2.16543988e+04  2.49427011e+03 -6.93040658e+04  1.03184829e+05
 -6.35604187e+04  1.47011138e+04]
4rib_X-4rib_Z DNA.txt
4rib_X-4rib_Z DNA.txt
0
1
[-3.46000000e+01 -1.93358839e+02  2.15914039e+03 -9.72985862e+03
  1.35105897e+04  2.46532196e+04 -1.03337078e+05  1.33634280e+05
 -7.85105876e+04  1.78354923e+04]
4ric_T-4ric_V DNA.txt
4ric_T-4ric_V DNA.txt
0
1
[ 1.93500000e+01 -7.41516666e+01  2.06554970e+03 -2.02960551e+04
  7.70808264e+04 -1.29450336e+05  7.10067011e+04  5.76348742e+04
 -8.81679594e+04  3.